# 08 Classify videos by joke type.

Each video demonstrates a single joke type. We have this as meta-data. Can we train a classifier based on the movement data?

Each video shows one joke from a set of five possibilities [Peekaboo,TearingPaper,NomNomNom,ThatsNotAHat,ThatsNotACat].

We will use TensorFlow to train a classifier to predict the joke type from the movement data.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# for each row of processedVideos, we add one timeseries to the dataset

def create_movementdataset(processedVideos):
    
    
    dataset = tf.
    for i in range(len(processedVideos)):
        dataset.append((processedVideos[i], labels[i]))
    return dataset